In [ ]:
import pandas as pd

# 0
transactions = pd.read_csv("/content/drive/MyDrive/ITMO/5-sem/ml/transactions.csv", nrows=1000000, index_col='customer_id')
tr_types = pd.read_csv("/content/drive/MyDrive/ITMO/5-sem/ml/tr_types.csv", sep=";", index_col='tr_type')
tr_mcc_codes = pd.read_csv("/content/drive/MyDrive/ITMO/5-sem/ml/tr_mcc_codes.csv", sep=";", index_col='mcc_code')
gender_train = pd.read_csv("/content/drive/MyDrive/ITMO/5-sem/ml/gender_train.csv", sep=",", index_col='customer_id')

print(transactions.head())
print(tr_types.head())
print(tr_mcc_codes.head())
print(gender_train.head())

            tr_datetime  mcc_code  tr_type    amount term_id
customer_id                                                 
39026145     0 10:23:26      4814     1030  -2245.92     NaN
39026145     1 10:19:29      6011     7010  56147.89     NaN
39026145     1 10:20:56      4829     2330 -56147.89     NaN
39026145     1 10:39:54      5499     1010  -1392.47     NaN
39026145     2 15:33:42      5499     1010   -920.83     NaN
                                            tr_description
tr_type                                                   
3200     Плата за предоставление услуг посредством моби...
3210     Плата за предоставление отчета по счету карты ...
3800     Плата за обслуживание банковской карты (за пер...
4000               Плата за получение наличных в Сбербанке
4001     Плата за получение наличных в Сбербанке (в дру...
                                            mcc_description
mcc_code                                                   
742                                     

In [ ]:
# 1.1
"""Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample"""
sample = transactions["tr_type"].sample(1000)

# 1.2
"""В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM"""
descriptions = tr_types['tr_description']
filtered_types = tr_types[descriptions.str.contains('POS|АТМ')]
merge = sample[sample.isin(filtered_types.index)]


transaction_ratio = len(merge) / len(sample)
print(transaction_ratio)

0.653


In [ ]:
# 2.1
"""Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions."""
transaction_counts = transactions['tr_type'].value_counts(normalize=True)
print(transaction_counts)

# 2.2
"""Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже)."""

top_10_transactions = transaction_counts.head(10)
top_10_descriptions = tr_types.loc[top_10_transactions.index]['tr_description']

result = pd.DataFrame({'tr_type': top_10_transactions.index, 'tr_description': top_10_descriptions.values})
print(result)

1010    0.231117
2010    0.151166
7070    0.149006
1110    0.137658
1030    0.118975
          ...   
2446    0.000004
4096    0.000004
8146    0.000003
1510    0.000002
8100    0.000002
Name: tr_type, Length: 72, dtype: float64
   tr_type                                     tr_description
0     1010                              Покупка. POS ТУ СБ РФ
1     2010              Выдача наличных в АТМ Сбербанк России
2     7070  Перевод на карту (с карты) через Мобильный бан...
3     1110                             Покупка. POS ТУ Россия
4     1030                     Оплата услуги. Банкоматы СБ РФ
5     2370  Списание с карты на карту по операции <перевод...
6     7010       Взнос наличных через АТМ (в своем тер.банке)
7     7030  Перевод на карту (с карты) через АТМ (в предел...
8     7071  Перевод на карту (с карты) через Мобильный бан...
9     1100                                Покупка. ТУ  Россия


In [ ]:
# 3.1
"""В датафрейме transactions найдите клиента с максимальной суммой приходов на карту"""
customer_with_max_income = transactions[transactions['amount'] > 0].groupby('customer_id')['amount'].sum().idxmax()

# 3.2
"""В датафрейме transactions найдите клиента с максимальной суммой расходов по карте"""
customer_with_max_expenses = transactions[transactions['amount'] < 0].groupby('customer_id')['amount'].sum().idxmin()

# 3.3
"""Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов"""
transactions.loc[customer_with_max_income, 'amount']
# 3.3
"""Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов"""
income_total = transactions.loc[(transactions.index == customer_with_max_income) & (transactions['amount'] > 0), 'amount'].sum()
expenses_total = transactions.loc[(transactions.index == customer_with_max_expenses) & (transactions['amount'] < 0), 'amount'].sum()
difference = income_total + abs(expenses_total)

print("Customer with max income:", customer_with_max_income)
print("Customer with max expenses:", customer_with_max_expenses)
print("Difference:", difference)

Customer with max income: 70780820
Customer with max expenses: 70780820
Difference: 2498067091.6


In [ ]:
# 4.1
"""Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2"""
print(result)
part1 = transactions[transactions["tr_type"].isin(result.index)].groupby("tr_type")
print(part1['amount'].mean)

# 4.2
"""Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3"""
customer_3_amounts = transactions[transactions.isin([customer_with_max_income, customer_with_max_expenses])]['amount']
mean_amount_customer_3 = customer_3_amounts.mean()
median_amount_customer_3 = customer_3_amounts.median()

print("Mean amount for customer from task 3:", mean_amount_customer_3)
print("Median amount for customer from task 3:", median_amount_customer_3)

   tr_type                                     tr_description
0     1010                              Покупка. POS ТУ СБ РФ
1     2010              Выдача наличных в АТМ Сбербанк России
2     7070  Перевод на карту (с карты) через Мобильный бан...
3     1110                             Покупка. POS ТУ Россия
4     1030                     Оплата услуги. Банкоматы СБ РФ
5     2370  Списание с карты на карту по операции <перевод...
6     7010       Взнос наличных через АТМ (в своем тер.банке)
7     7030  Перевод на карту (с карты) через АТМ (в предел...
8     7071  Перевод на карту (с карты) через Мобильный бан...
9     1100                                Покупка. ТУ  Россия
<bound method GroupBy.mean of <pandas.core.groupby.generic.SeriesGroupBy object at 0x7f3424217490>>
Mean amount for customer from task 3: nan
Median amount for customer from task 3: nan


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [ ]:
top_10_transactions = transaction_counts.head(10) # из задания 2.1
mean_top_10 = transactions[transactions['tr_type'].isin(top_10_transactions.index)]['amount'].mean()
median_top_10 = transactions[transactions['tr_type'].isin(top_10_transactions.index)]['amount'].median()

print("Среднее значение по всем типам транзакций из топ 10:", mean_top_10)
print("Медиана по всем типам транзакций из топ 10:", median_top_10)
